In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
con = sqlite3.connect(f"../data/chembl_29.db")
df = pd.read_sql_query("SELECT compound_properties.*, molecule_dictionary.chembl_id, compound_structures.canonical_smiles FROM compound_properties INNER JOIN compound_structures ON compound_structures.molregno=compound_properties.molregno INNER JOIN molecule_dictionary ON molecule_dictionary.molregno=compound_properties.molregno;", con)

In [23]:
acti_sup = pd.read_sql("select * from activities", con)

In [6]:
df

,molregno,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,...,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,chembl_id,canonical_smiles
0,1,341.75,2.11,5.0,1.0,84.82,3.0,N,0.0,6.48,...,3.0,24.0,0.74,341.0567,C17H12ClN3O3,6.0,1.0,0.0,CHEMBL6329,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl
1,2,332.32,1.33,6.0,1.0,108.61,3.0,N,0.0,6.33,...,3.0,25.0,0.73,332.0909,C18H12N4O3,7.0,1.0,0.0,CHEMBL6328,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1
2,3,357.80,2.27,5.0,2.0,87.98,3.0,N,0.0,6.33,...,3.0,25.0,0.75,357.0880,C18H16ClN3O3,6.0,2.0,0.0,CHEMBL265667,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1
3,4,307.31,1.46,5.0,1.0,84.82,3.0,N,0.0,6.33,...,3.0,23.0,0.74,307.0957,C17H13N3O3,6.0,1.0,0.0,CHEMBL6362,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1
4,5,341.75,2.11,5.0,1.0,84.82,3.0,N,0.0,6.33,...,3.0,24.0,0.74,341.0567,C17H12ClN3O3,6.0,1.0,0.0,CHEMBL267864,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084719,2487510,556.38,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,...,NaN,NaN,NaN,556.2055,C31H32BNO6P+,NaN,NaN,NaN,CHEMBL4651250,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...
2084720,2487511,378.30,3.13,3.0,2.0,58.36,5.0,N,0.0,NaN,...,2.0,25.0,0.84,377.1062,C19H21Cl2N3O,4.0,3.0,0.0,CHEMBL4651251,CNCc1ccc2c(c1)CN(C(=O)[C@H](N)Cc1ccc(Cl)cc1Cl)C2
2084721,2487512,356.42,2.29,5.0,4.0,90.82,7.0,N,0.0,9.18,...,2.0,26.0,0.57,356.1736,C20H24N2O4,6.0,4.0,0.0,CHEMBL4651252,Cc1ccc(CCCNCC(O)c2cc(O)cc3c2OCC(=O)N3)cc1
2084722,2487513,1023.94,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,...,NaN,NaN,NaN,1023.2919,C42H61FN3O17P3S,NaN,NaN,NaN,CHEMBL4651253,CCCCCCCCCCCCCCCCOC(=O)Oc1ccc(COP(=O)(OCc2ccc(O...


In [24]:
acti_sup

,activity_id,assay_id,doc_id,record_id,molregno,standard_relation,standard_value,standard_units,standard_flag,standard_type,...,toid,upper_value,standard_upper_value,src_id,type,relation,value,units,text_value,standard_text_value
0,31863,54505,6424,206172,180094,>,100000.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,>,100.0000,uM,None,None
1,31864,83907,6432,208970,182268,=,2500.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,=,2.5000,uM,None,None
2,31865,88152,6432,208970,182268,>,50000.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,>,50.0000,uM,None,None
3,31866,83907,6432,208987,182855,=,9000.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,=,9.0000,uM,None,None
4,31867,88153,6432,208987,182855,None,NaN,None,0,IC50,...,NaN,NaN,None,1,IC50,None,NaN,uM,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18635911,22070016,1998352,116894,3489675,1625905,=,25000.0,nM,1,Kd,...,NaN,NaN,None,55,Kd,=,25.0000,uM,None,None
18635912,22070017,1998353,116894,3489675,1625905,=,8474.6,nM,1,Kd,...,NaN,NaN,None,55,Kd,=,8.4746,uM,None,None
18635913,22070018,1998354,116888,3489679,1966054,=,31847133.8,nM,1,Kd,...,NaN,NaN,None,55,Kd,=,31847.1338,uM,None,None
18635914,22070019,1998347,116887,3489682,2486473,=,48.0,nM,1,Kd,...,NaN,NaN,None,55,Kd,=,0.0480,uM,None,None


In [25]:
acti_sup.standard_type.unique()

array(['IC50', '% remaining', 'Delta S-H', ..., 'MEC2.0',
       'Percent Effect', 'Delta TM'], dtype=object)

In [26]:
acti_sup.loc[acti_sup.standard_type == "IC50"]

,activity_id,assay_id,doc_id,record_id,molregno,standard_relation,standard_value,standard_units,standard_flag,standard_type,...,toid,upper_value,standard_upper_value,src_id,type,relation,value,units,text_value,standard_text_value
0,31863,54505,6424,206172,180094,>,100000.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,>,100.0000,uM,None,None
1,31864,83907,6432,208970,182268,=,2500.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,=,2.5000,uM,None,None
2,31865,88152,6432,208970,182268,>,50000.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,>,50.0000,uM,None,None
3,31866,83907,6432,208987,182855,=,9000.0,nM,1,IC50,...,NaN,NaN,None,1,IC50,=,9.0000,uM,None,None
4,31867,88153,6432,208987,182855,None,NaN,None,0,IC50,...,NaN,NaN,None,1,IC50,None,NaN,uM,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18635893,22069998,1998337,116894,3489675,1625905,=,116.1,nM,1,IC50,...,NaN,NaN,None,55,IC50,=,0.1161,uM,None,None
18635894,22069999,1998338,116894,3489675,1625905,=,47.0,nM,1,IC50,...,NaN,NaN,None,55,IC50,=,0.0470,uM,None,None
18635895,22070000,1998338,116901,3489685,2205530,=,49.2,nM,1,IC50,...,NaN,NaN,None,55,IC50,=,0.0492,uM,None,None
18635896,22070001,1998338,116895,3489680,2486472,=,189.0,nM,1,IC50,...,NaN,NaN,None,55,IC50,=,0.1890,uM,None,None
